In [24]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
pd.set_option('max_columns', 20)
pd.set_option('max_columns', None)

# pd.set_option('max_rows', 500)

In [48]:
data = pd.read_csv('../csv/dataClean_forPrice_20210719.csv', index_col=0, encoding='utf-8')

In [56]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54150 entries, 0 to 54149
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   car_brand    54150 non-null  int64
 1   car_year     54150 non-null  int64
 2   car_mileage  54150 non-null  int64
 3   car_price    54150 non-null  int64
dtypes: int64(4)
memory usage: 2.1 MB


In [55]:
data.drop(columns=['source', 'car_model'], inplace=True)

In [35]:
np.set_printoptions(threshold=len(data['car_model'].unique()))
print(data['car_model'].unique())
print(data['car_brand'].unique())
print(len(data['car_model'].unique()))
print(data['car_brand'].unique())
print(len(data['car_brand'].unique()))

['ALTIS' 'DELICA' 'CARRY' 'MARCH' 'SENTRA' 'KUGA' 'FOCUS 五門' 'ZINGER'
 'CR-V' 'LIVINA' 'TIIDA 5門' 'CAMRY' 'WISH' 'VIOS' 'MAZDA 5' 'FORTIS'
 'VERYCA' 'MAZDA 3' 'AURIS' 'COLT PLUS' 'KICKS' 'HR-V' 'SIENTA' 'FIT'
 'SANTA FE' 'CITY' 'TEANA' 'YARIS' 'S5 TURBO' 'SX4' 'RAV4' 'INNOVA'
 'FOCUS' 'OUTLANDER' 'MAZDA3 5門' 'TUCSON' 'IX35' 'TIIDA' 'PREVIA'
 'X-TRAIL' 'CX-3' 'CIVIC' 'FIESTA' 'LANCER IO' 'U6 TURBO' 'ACCORD' 'C-HR'
 'ELANTRA' 'ODYSSEY' 'FORESTER' 'SERENA Q-RV' 'SWIFT' 'SIENNA' 'CX-5'
 'CX-9' 'MAZDA 2' 'FOCUS ST' 'KONA' 'STAREX' 'LEVORG' 'IS200t' 'U5' 'JUKE'
 'FOCUS 四門' 'I30' 'PORTER' 'ESCAPE' 'M7 TURBO' 'SAVRIN' 'CAMRY HYBRID'
 'LANCER FORTIS' 'VITARA' 'CANTER' '7 SUV' 'ALPHARD' '7 MPV' 'ESCORT'
 'RX200t' 'OUTBACK' 'MAZDA6' 'TIIDA 4門' 'MATRIX' 'MAZDA 6' 'MONDEO'
 '5 SEDAN' 'ECONOVAN' 'PRIUS C' 'GRUNDER' 'NX200t' 'JIMNY' 'MUSTANG'
 'U7 TURBO' 'S3' 'NX200' 'RX300' 'IGNIS' 'I-MAX' 'XV' 'ECLIPSE' 'UX200'
 'PRIUS' 'RANGER' 'LANCER' 'TACOMA' 'WRX' 'SUPRA' '300' 'I30 CW' 'TOURNEO'
 'I10' 'ES250

In [49]:
data.loc[data['car_brand'].str.contains('BENZ') == True, 'car_brand'] = 'BENZ'

In [34]:
data.loc[data['car_brand'].str.contains('BENZ') == True, 'car_brand']

1814     BENZ
1816     BENZ
1822     BENZ
1823     BENZ
1825     BENZ
         ... 
54130    BENZ
54140    BENZ
54142    BENZ
54143    BENZ
54147    BENZ
Name: car_brand, Length: 8921, dtype: object

In [50]:
car_brand_dict = {k:v for v,k in enumerate(data['car_brand'].unique())}

In [51]:
print(car_brand_dict)

{'TOYOTA': 0, 'MITSUBISHI': 1, 'SUZUKI': 2, 'NISSAN': 3, 'FORD': 4, 'HONDA': 5, 'MAZDA': 6, 'HYUNDAI': 7, 'LUXGEN': 8, 'SUBARU': 9, 'LEXUS': 10, 'HINO': 11, 'KIA': 12, 'BMW': 13, 'VOLKSWAGEN': 14, 'LAND ROVER': 15, 'BENZ': 16, 'SKODA': 17, 'MINI': 18, 'VOLVO': 19, 'AUDI': 20, 'SMART': 21, 'PEUGEOT': 22, 'PORSCHE': 23, 'CITROEN': 24, 'INFINITI': 25, 'JAGUAR': 26, 'MCC': 27, 'SSANGYONG': 28, 'MAHINDRA': 29, 'ISUZU': 30, 'LAMBORGHINI': 31, 'DFSK': 32, 'FERRARI': 33, 'ALFA ROMEO': 34, 'CHEVROLET': 35, 'DODGE': 36, 'DAIHATSU': 37, 'MASERATI': 38, 'PROTON': 39, 'OPEL': 40, 'FOTON': 41, 'TOBE': 42, 'SAAB': 43, 'FIAT': 44, 'JEEP': 45, 'BENTLEY': 46, 'VW': 47, 'LAND': 48, 'MCLAREN': 49, 'CHRYSLER': 50, 'TESLA': 51, 'BUICK': 52, 'CMC': 53, 'ASTON MARTIN': 54, 'AUSTIN': 55, 'HUMMER': 56, 'RENAULT': 57, 'MG': 58, 'ROLLS-ROYCE': 59, 'SCION': 60, 'ACURA': 61, 'CADILLAC': 62, 'IVECO': 63, 'FORMOSA': 64, 'LOTUS': 65, 'ASIA': 66}


In [52]:
data['car_brand'] = data['car_brand'].map(car_brand_dict)

In [82]:
data

,car_brand,car_year,car_mileage,car_price
0,0,2009,197710,196000
1,1,2016,81609,358000
2,2,2013,53167,286000
3,3,2013,92757,238000
4,3,2015,43307,378000
...,...,...,...,...
54145,5,2007,280000,168000
54146,0,2018,30000,398000
54147,16,2012,184000,720000
54148,8,2019,23000,260000


In [109]:
X = data.drop(['car_price'], axis = 1)
y = data.loc[:,'car_price']

In [98]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [99]:
X

array([[-1.06884568, -0.99113543, -0.00242202],
       [-0.96056719,  0.54337622, -0.01004543],
       [-0.85228869, -0.11427163, -0.01191298],
       ...,
       [ 0.66361018, -0.33348758, -0.00332224],
       [-0.20261775,  1.20102408, -0.0138938 ],
       [ 0.3387747 , -0.11427163, -0.01146431]])

In [100]:
X_train , X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=34)

# Training

In [101]:
lin_model = LinearRegression()
lin_model.fit(X_train, y_train)
lin_y_preds = lin_model.predict(X_test)

In [102]:
lgb_model = lgb.LGBMRegressor(
    boosting_type='gbdt',
    num_leaves=31,
    reg_lambda=1.0
)

lgb_model.fit(X_train, y_train)
lgb_y_preds = lgb_model.predict(X_test)

In [103]:
lin_loss = np.sqrt(mean_squared_error(y_test, lin_y_preds))

lgb_loss = np.sqrt(mean_squared_error(y_test, lgb_y_preds))

In [104]:
print('Linear Regression RMSE：',lin_loss)
print('Gradient Boosted RMSE：', lgb_loss)

Linear Regression RMSE： 597253.8421180319
Gradient Boosted RMSE： 547943.6294399801


In [105]:
print('Linear Regression R*2：', lin_model.score(X_test, y_test))
print('Gradient Boosted R*2：', lgb_model.score(X_test, y_test))

Linear Regression R*2： 0.18791056858367572
Gradient Boosted R*2： 0.31646978310367135


In [106]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

tree = DecisionTreeClassifier()
bag = BaggingClassifier(tree, n_estimators=100, max_samples=0.8,
                        random_state=0)
bag.fit(X_train, y_train)
print('DecisionTree R*2：', bag.score(X_test, y_test))

DecisionTree R*2： 0.14459833795013852


In [107]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(n_estimators=100, random_state=0)
RF.fit(X_train, y_train)
print('RandomForest R*2：', RF.score(X_test, y_test))

RandomForest R*2： 0.14644506001846722
